# CS549 Machine Learning
# Assignment 7: Optimization of Deep Neural Networks

**Author:** Yang Xu, Assistant Professor of Computer Science, San Diego State University

**Total points: 15**

In this assignment, you will implement a multiple layer feed-forward neural network for a multi-class classification task.

In [19]:
import torch
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Task 1: Build a deep neural network model
**Points: 3**

Implement the `NeuralNetModel1` class. The model takes a $28\times 28$ grey-scale image as input, and pass it through a deep neural network.

The network has 2 hidden layers and 1 output layers, whose sizes are: 512 -> 512 -> 10. That is, the number of output classes is 10. The activation function for each hidden layer is `ReLU`.

The input image is first passed through a `nn.Flatten()` layer so that a 2D tensor becomes 1D.

In [20]:
class NeuralNetModel1(nn.Module):
    def __init__(self):
        super(NeuralNetModel1, self).__init__()
        ### START YOUR CODE ###
        self.flatten = nn.Flatten() # Use nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512), # Input size is 28*28
            nn.ReLU(), # ReLU
            nn.Linear(512, 512), # 512 -> 512
            nn.ReLU(), # ReLU
            nn.Linear(512, 10), # 512 -> 10
        )
        ### END YOUR CODE ###

    def forward(self, x):
        ### START YOUR CODE ###
        x = self.flatten(x) # Call self.flatten()
        logits = self.linear_relu_stack(x) # Call self.linear_relu_stack()
        ### END YOUR CODE ###

        return logits

In [21]:
# Do not change the test code here
sample_input = torch.randn(5, 28, 28)
print('input size:', sample_input.size())

model1 = NeuralNetModel1()
with torch.no_grad():
    output = model1(sample_input)
print('output size:', output.size())

input size: torch.Size([5, 28, 28])
output size: torch.Size([5, 10])


**Expected output**:

input size: torch.Size([5, 28, 28])\
output size: torch.Size([5, 10])

---

## Task 2: Use dataloader
**Points: 1**

Download the FashionMNIST dataset provided by PyTorch to the folder "data", which takes some time for the first time execution.
Use the `DataLoader` module to wrap the loaded training and test data. Specify the `batch_size` correctly for both training and test dataloader.

See <https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader> for more information.

In [22]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True, # True
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False, # False
    download=True,
    transform=ToTensor()
)

batch_size = 64

### START YOUR CODE ###
train_loader = DataLoader(training_data, batch_size=batch_size) # Specify data source and batch size correctly
test_loader = DataLoader(test_data, batch_size=batch_size)
### END YOUR CODE ###

In [23]:
# Do not change the test code here
print('Training data size:', len(training_data))
print('Testing data size:', len(test_data))

count = 0
for batch in train_loader:
    X, y = batch
    print('X size:', X.size())
    print('y size:', y.size())
    count += 1
    if count > 0:
        break

Training data size: 60000
Testing data size: 10000
X size: torch.Size([64, 1, 28, 28])
y size: torch.Size([64])


**Expected output**:

Training data size: 60000\
Testing data size: 10000\
X size: torch.Size([64, 1, 28, 28])\
y size: torch.Size([64])

## Task 3: Define loss and optimizer
**Points: 1**

Use `nn.CrossEntropyLoss()` as the loss function, and use `torch.optim.SGD()` as the optimizer. Specify the arguments for `SGD()`, including the learning rate correctly.

See <https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html> and <https://pytorch.org/docs/stable/optim.html> for more information.

In [24]:
learning_rate = 1e-3

### START YOUR CODE ###
loss_fn = nn.CrossEntropyLoss()
optimizer_sgd = torch.optim.SGD(params=model1.parameters(), lr=learning_rate)
### END YOUR CODE ###

In [25]:
# Do not change the test code here
print(loss_fn)
print(type(optimizer_sgd))

CrossEntropyLoss()
<class 'torch.optim.sgd.SGD'>


**Expected output**:

CrossEntropyLoss()
<class 'torch.optim.sgd.SGD'>

---

## Task 4: Implement train and test functions
**Points: 6**

Implement the code for training the model in `train()`. Implement the code for testing the model in `test()`. For the backpropagation step, you need to first zero out all gradients by calling `optimizer.zero_grad()` before carrying out `backward()` and `step()` to update parameters.

In `test()`, you need to calculate the number of correct prediction in the current batch, and add it to the `correct` variable.
Finally, you need to divide `correct` by the total number of test examples to obtain the test accuracy.

In [26]:
def train_loop(dataloader, model, loss_fn, optimizer, verbose=True):
    for i, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        ### START YOUR CODE ###
        pred = model(X) # Get the prediction output from model
        loss = loss_fn(pred, y) # compute loss by calling loss_fn()
        ### END YOUR CODE ###

        # Backpropagation
        ### START YOUR CODE ###
        optimizer.zero_grad() # zero_grad()
        loss.backward() # backward()
        optimizer.step() # step()
        ### END YOUR CODE ###

        if verbose and i % 100 == 0:
            loss = loss.item()
            current_step = i * len(X)
            print(f"loss: {loss:>7f}  [{current_step:>5d}/{len(dataloader.dataset):>5d}]")

In [27]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn):
    test_loss, correct = 0, 0

    for X, y in dataloader:
        ### START YOUR CODE ###
        pred = model(X) # Similar to how it is computed in train()
        loss = loss_fn(pred, y)
        test_loss += loss.item()
        correct += torch.mean((torch.argmax(pred, dim=1) == y).float()) # Add the number of correct prediction in the current batch to `correct`
        ### END YOUR CODE ###

    test_loss /= len(dataloader)
    ### START YOUR CODE ###
    test_acc = correct / len(dataloader) # Use `correct` to compute accuracy
    ### END YOUR CODE ###

    print(f"Test Error: \n Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Next, execute the following cell to start the training and testing loop. Make sure that the cell containing the loss function and optimizers has already been executed.

In [28]:
model1 = NeuralNetModel1() # Reset the model
### START YOUR CODE ###
optimizer_sgd = torch.optim.SGD(params=model1.parameters(), lr=learning_rate) # Because the model1 is reset, the optimizer also needs redefined.
### END YOUR CODE ###

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    ### START YOUR CODE ###
    train_loop(train_loader, model1, loss_fn, optimizer_sgd, verbose=False) # Use verbose=False, if you want to see less information
    test_loop(test_loader, model1, loss_fn)
    ### END YOUR CODE ###

print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 40.0%, Avg loss: 2.171464 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.913630 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.542018 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 63.2%, Avg loss: 1.270237 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 64.6%, Avg loss: 1.102932 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 65.8%, Avg loss: 0.995855 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.922173 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 68.1%, Avg loss: 0.868536 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 69.2%, Avg loss: 0.827641 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 70.5%, Avg loss: 0.795204 

Done!


**Expected output**

The test accuracy from the last epoch should be above 70%.

---

Next, train an ADAM optimizer. Note that the model needs be reset.

In [29]:
model1 = NeuralNetModel1() # Reset the model

### START YOUR CODE ###
optimizer_adam = torch.optim.Adam(params=model1.parameters(), lr=learning_rate)
### END YOUR CODE ###

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    ### START YOUR CODE ###
    train_loop(train_loader, model1, loss_fn, optimizer_adam, verbose=False) # Use verbose=False, if you want to see less information
    test_loop(test_loader, model1, loss_fn)
    ### END YOUR CODE ###

print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.422074 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 85.7%, Avg loss: 0.386433 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 85.9%, Avg loss: 0.383809 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.364839 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.342775 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.342688 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.342064 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 88.2%, Avg loss: 0.350175 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.367938 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 88.4%, Avg loss: 0.347578 

Done!


**Expected output**:

You can find that the training converges much faster using ADAM.

---

## Task 5: Add batchnorm and dropout
**Points: 4**

Use `torch.nn.BatchNorm1d()` and `nn.Dropout()` after the ReLU activation of each hidden layer. `Batchnorm1d()` takes the size of previous activation as input. `Dropout()` takes the probability of dropout as input.

For more information, see <https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html> and <https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html>.

In [30]:
class NeuralNetModel2(nn.Module):
    def __init__(self, dropout = 0.1): # Note the additional dropout parameter here
        """
        :param dropout: float, the probability of dropout
        """
        super(NeuralNetModel2, self).__init__()
        ### START YOUR CODE ###
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(), # ReLU
            nn.BatchNorm1d(512), # Batchnorm
            nn.Dropout(p=dropout), # Dropout, use the `dropout` parameter

            nn.Linear(512, 512),
            nn.ReLU(), # ReLU
            nn.BatchNorm1d(512), # Batchnorm
            nn.Dropout(p=dropout), # Dropout, use the `dropout` parameter

            nn.Linear(512, 10),
        )
        ### END YOUR CODE ###

    def forward(self, x):
        ### START YOUR CODE ###
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        ### END YOUR CODE ###

        return logits

In the following cell, test with different `dropout` rates, and observe how that affects the test accuracy.

In [36]:
### START YOUR CODE ###
model2 = NeuralNetModel2(dropout=0.5) # Call NeuralNetModel2() with the dropout value you want to try
optimizer = torch.optim.Adam(params=model2.parameters(), lr=learning_rate) # You may try Adam/SGD optimizer
### END YOUR CODE ###

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    ### START YOUR CODE ###
    train_loop(train_loader, model2, loss_fn, optimizer, verbose=False) # Use verbose=False, if you want to see less information
    test_loop(test_loader, model2, loss_fn)
    ### END YOUR CODE ###

print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 81.1%, Avg loss: 0.520235 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 82.2%, Avg loss: 0.498800 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 82.9%, Avg loss: 0.483629 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 82.6%, Avg loss: 0.486645 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.464803 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.452396 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 83.4%, Avg loss: 0.459188 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 83.9%, Avg loss: 0.461147 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.437681 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 84.6%, Avg loss: 0.438506 

Done!


**Expected output**

In theory, you should see that the larger dropout rate you use, the lower test accuracy you will get, at the same epoch number.

But the model trained with some dropout rate should generalize better to new data.